# Build a list of sources and run photoz

In [ ]:
import os
import lephare as lp
import numpy as np
from matplotlib import pylab as plt

%matplotlib inline

In [ ]:
config_file = "./data/COSMOS.para"
keymap = lp.read_config(config_file)
lp.data_retrieval.get_auxiliary_data(keymap=keymap, additional_files=["examples/COSMOS.in"])

In [ ]:
keymap["ZPHOTLIB"] = lp.keyword("ZPHOTLIB", "VISTA_COSMOS_FREE,ALLSTAR_COSMOS,QSO_COSMOS")
keymap["CAT_IN"] = lp.keyword("CAT_IN", "$LEPHAREDIR/examples/COSMOS.in")
keymap["CAT_OUT"] = lp.keyword("CAT_OUT", "zphot_short.out")
keymap["ADD_EMLINES"] = lp.keyword("ADD_EMLINES", "0,100")
keymap["AUTO_ADAPT"] = lp.keyword("ADD_EMLINES", "YES")
keymap["Z_STEP"] = lp.keyword("Z_STEP", "0.04,0,6")
keymap["CAT_LINES"] = lp.keyword("CAT_LINES", "1,100")
keymap["SPEC_OUT"] = lp.keyword("SPEC_OUT", "YES")
keymap["PARA_OUT"] = lp.keyword("PARA_OUT", "./data/output.para")
keymap["VERBOSE"] = lp.keyword("VERBOSE", "NO")
keymap["ZFIX"] = lp.keyword("ZFIX", "NO")

In [ ]:
# before running PhotoZ we must run filt, sedtolib and maggal
filterLib = lp.Filter(config_file=config_file)
filterLib.run()
sedlib = lp.Sedtolib(config_keymap=keymap)
sedlib.run(typ="STAR", star_sed="$LEPHAREDIR/sed/STAR/STAR_MOD_ALL.list")
sedlib.run(typ="QSO", qso_sed="$LEPHAREDIR/sed/QSO/SALVATO09/AGN_MOD.list", gal_lib="LIB_QSO")
sedlib.run(typ="GAL", gal_sed="$LEPHAREDIR/examples/COSMOS_MOD.list", gal_lib="LIB_GAL")
maglib = lp.MagGal(config_keymap=keymap)
maglib.run(typ="STAR", lib_ascii="YES", star_lib_out="ALLSTAR_COSMOS")
maglib.run(
    typ="QSO",
    lib_ascii="YES",
    mod_extinc="0,1000",
    eb_v="0.,0.1,0.2,0.3",
    extinc_law="SB_calzetti.dat",
    qso_lib_in="LIB_QSO",
)
maglib.run(
    typ="GAL",
    lib_ascii="YES",
    gal_lib_in="LIB_GAL",
    gal_lib_out="VISTA_COSMOS_FREE",
    mod_extinc="18,26,26,33,26,33,26,33",
    extinc_law="SMC_prevot.dat,SB_calzetti.dat,SB_calzetti_bump1.dat,SB_calzetti_bump2.dat",
    em_lines="EMP_UV",
    em_dispersion="0.5,0.75,1.,1.5,2.",
)

In [ ]:
photz = lp.PhotoZ(keymap)

In [ ]:
cat = np.loadtxt(f"{lp.LEPHAREDIR}/examples/COSMOS.in")

In [ ]:
zspec_mask = np.logical_and(cat[:, 62] > 0.01, cat[:, 62] < 6)
zs_cat = cat[zspec_mask]
spec = zs_cat[:, 0]
fluxes = zs_cat[:, 1:60:2]
efluxes = zs_cat[:, 2:61:2]
context = zs_cat[:, 61]
zspec = zs_cat[:, 62]

In [ ]:
keymap["ADAPT_LIM"], keymap["ADAPT_BAND"]

In [ ]:
mag = np.zeros_like(fluxes[:, 4]) + 1000.0
mag[fluxes[:, 4] > 0] = -2.5 * np.log10(fluxes[:, 4][fluxes[:, 4] > 0]) - 48.6
mag_mask = np.logical_and(mag > 1.5, mag < 20.5)
auto_adapt_data = zs_cat[mag_mask]

In [ ]:
srclist = []
nobj = 0
imagm = 30
gbcont = 0
contforb = -1
shifts0 = [0]
fac_err = [1.5]
min_err = [
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.05,
    0.05,
    0.05,
    0.05,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.05,
    0.05,
    0.05,
    0.05,
    0.1,
    0.2,
    0.3,
]
for d in auto_adapt_data:
    oneObj = lp.onesource(nobj, photz.gridz)
    oneObj.readsource(str(d[0]), d[1:60:2], d[2:61:2], int(d[61]), d[62], str(d[-1]))
    photz.prep_data(oneObj)
    if len(shifts0) == imagm:
        oneObj.adapt_mag(shifts0, shifts1)
    srclist.append(oneObj)
    nobj += 1

In [ ]:
len(srclist)

In [ ]:
a0, a1 = photz.run_autoadapt(srclist)

In [ ]:
photozlist = []
for d in cat:
    oneObj = lp.onesource(nobj, photz.gridz)
    oneObj.readsource(str(d[0]), d[1:60:2], d[2:61:2], int(d[61]), d[62], str(d[-1]))
    photz.prep_data(oneObj)
    photozlist.append(oneObj)

In [ ]:
len(photozlist)

In [ ]:
photz.run_photoz(photozlist[:10], a0, a1)

In [ ]:
t = photz.build_output_tables(photozlist[:10], para_out="./data/output.para", filename="toto.fits")

In [ ]:
t